### Import packages

In [1]:
import cirq
import numpy as np
import pprint
from utilities.frac_check import find_closest

### Define gate library

#### NOTES
- How to decompose gates with parameters (if even possible)?:
   - Controlled-phase gate: CPhase(theta)
   - Barenco gate: Barenco(phi, alpha, theta)
   - Givens gate: Givens(theta)
   - Swap alpha gate: SwapAlpha(alpha)
   - parametric Swap gate: pSwap(theta)
   - A gate: A(theta, phi)
   - Fermionic Simulator: FSim(theta, phi)

- The parameterized Ising gates XX, YY, ZZ, XY do not require a decomposition
   (only nonlocal interactions).

- Following gates do not need to be decomposed (only nonlocal interactions):
   - Dagwood Bumstead gate (DB)
   - Squareroot of Swap (SqSwap)
   - Inverse squareroot of Swap (InvSqSwap)
   - Berkeley gate (B)
   - ECP gate (ECP)

In [2]:
gate_library = {
    "I2":
        np.array([[1, 0, 0, 0],
                  [0, 1, 0, 0],
                  [0, 0, 1, 0],
                  [0, 0, 0, 1]]),
    "CNOT":
        np.array([[1, 0, 0, 0],
                  [0, 1, 0, 0],
                  [0, 0, 0, 1],
                  [0, 0, 1, 0]]),
    "CY":
        np.array([[1, 0, 0, 0],
                  [0, 1, 0, 0],
                  [0, 0, 0, -1j],
                  [0, 0, 1j, 0]]),
    "CZ":
        np.array([[1, 0, 0, 0],
                  [0, 1, 0, 0],
                  [0, 0, 1, 0],
                  [0, 0, 0, -1]]),
    "CH":
        np.array([[1, 0, 0, 0],
                  [0, 1, 0, 0],
                  [0, 0, 1 / np.sqrt(2), 1 / np.sqrt(2)],
                  [0, 0, 1 / np.sqrt(2), -1 / np.sqrt(2)]]),
    "MS":
        1 / np.sqrt(2) * np.array([[1, 0, 0, 1j],
                                   [0, 1, 1j, 0],
                                   [0, 1j, 1, 0],
                                   [1j, 0, 0, 1]]),
    "M":
        1 / np.sqrt(2) * np.array([[1, 1j, 0, 0],
                                   [0, 0, 1j, 1],
                                   [0, 0, 1j, -1],
                                   [1, -1j, 0, 0]]),
    "iSWAP":
        np.array([[1, 0, 0, 0],
                  [0, 0, 1j, 0],
                  [0, 1j, 0, 0],
                  [0, 0, 0, 1]]),
    "fSWAP":
        np.array([[1, 0, 0, 0],
                  [0, 0, 1, 0],
                  [0, 1, 0, 0],
                  [0, 0, 0, -1]]),
    "DCNOT":
        np.array([[1, 0, 0, 0],
                  [0, 0, 0, 1],
                  [0, 1, 0, 0],
                  [0, 0, 1, 0]]),
    "SWAP":
        np.array([[1, 0, 0, 0],
                  [0, 0, 1, 0],
                  [0, 1, 0, 0],
                  [0, 0, 0, 1]]),
    "CV":
        np.array([[1, 0, 0, 0],
                  [0, 1, 0, 0],
                  [0, 0, (1 + 1j) / 2, (1 - 1j) / 2],
                  [0, 0, (1 - 1j) / 2, (1 + 1j) / 2]]),
    "QFT2":
        1 / 2 * np.array([[1, 1, 1, 1],
                          [1, 1j, -1, -1j],
                          [1, -1, 1, -1],
                          [1, -1j, -1, 1j]]),
    "W":
        np.array([[1, 0, 0, 0],
                  [0, 1 / np.sqrt(2), 1 / np.sqrt(2), 0],
                  [0, 1 / np.sqrt(2), -1 / np.sqrt(2), 0],
                  [0, 0, 0, 1]]),
    "Syc":
        np.array([[1, 0, 0, 0],
                  [0, 0, -1j, 0],
                  [0, -1j, 0, 0],
                  [0, 0, 0, np.exp(-1j * np.pi / 6)]]),
}



### Define `Gate` class

In [3]:
class Gate:
    def __init__(self, name, matrix):
        self._name = name
        self._matrix = matrix

    @property
    def name(self):
        return self._name

    @name.setter
    def name(self, value):
        self._name = value

    @property
    def matrix(self):
        return self._matrix

    @matrix.setter
    def matrix(self, value):
        self._matrix = value

    def kak_decomposition(self):
        """
        U = g · (a1 ⊗ a0) · exp(i·pi·(x·XX + y·YY + z·ZZ)) · (b1 ⊗ b0)
        before: (b1 ⊗ b0)
        after:  (a1 ⊗ a0)

        t_x = x / 2
        t_y = y / 2
        t_z = z / 2

        K1 = b0
        K2 = b1
        K3 = a0
        K4 = a1

        """
        decomp = cirq.kak_decomposition(self._matrix)
        x, y, z = decomp.interaction_coefficients
        b0, b1 = decomp.single_qubit_operations_before
        a0, a1 = decomp.single_qubit_operations_after
        global_phase = decomp.global_phase

        t_x, t_y, t_z = (x * 2 / np.pi, y * 2 / np.pi, z * 2 / np.pi)

        b0a = cirq.axis_angle(b0)
        b1a = cirq.axis_angle(b1)
        a0a = cirq.axis_angle(a0)
        a1a = cirq.axis_angle(a1)

        self.decomposition = {
            "name": self._name,
            "matrix": self._matrix,
            "coefficients":
                {
                    "t_x": t_x,
                    "t_y": t_y,
                    "t_z": t_z,
                },
            "global phase": global_phase,
            "K1":
                {
                    "matrix": b0,
                    "angle (π)": b0a.angle / np.pi,
                    "angle (frac)": find_closest(value=b0a.angle / np.pi),
                    "axis": b0a.axis,
                    "global_phase": b0a.global_phase,
                },
            "K2":
                {
                    "matrix": b1,
                    "angle (π)": b1a.angle / np.pi,
                    "angle (frac)": find_closest(value=b1a.angle / np.pi),
                    "axis": b1a.axis,
                    "global_phase": b1a.global_phase,
                },
            "K3":
                {
                    "matrix": a0,
                    "angle (π)": a0a.angle / np.pi,
                    "angle (frac)": find_closest(value=a0a.angle / np.pi),
                    "axis": a0a.axis,
                    "global_phase": a0a.global_phase,
                },
            "K4":
                {
                    "matrix": a1,
                    "angle (π)": a1a.angle / np.pi,
                    "angle (frac)": find_closest(value=a1a.angle / np.pi),
                    "axis": a1a.axis,
                    "global_phase": a1a.global_phase,
                },
        }

### Print decompositions for gates in the gate library

In [4]:
for name, matrix in gate_library.items():
    g = Gate(name=name, matrix=matrix)
    g.kak_decomposition()
    print()
    print(f"===== {name} =====")
    print()
    pprint.pprint(g.decomposition)
    print()


===== I2 =====

{'K1': {'angle (frac)': (1, 2, 0.5, 0.5, 0.0),
        'angle (π)': 0.5,
        'axis': (-0.0, -0.0, 1.0),
        'global_phase': (1+0j),
        'matrix': array([[0.70710678-0.70710678j, 0.        +0.j        ],
       [0.        +0.j        , 0.70710678+0.70710678j]])},
 'K2': {'angle (frac)': (0, 1, 0.0, 0.0, 0.0),
        'angle (π)': 0.0,
        'axis': (1, 0, 0),
        'global_phase': (1+0j),
        'matrix': array([[1.+0.j, 0.+0.j],
       [0.+0.j, 1.+0.j]])},
 'K3': {'angle (frac)': (1, 2, -0.5, -0.5, 0.0),
        'angle (π)': -0.5,
        'axis': (0.0, 0.0, 1.0),
        'global_phase': (1+0j),
        'matrix': array([[0.70710678+0.70710678j, 0.        +0.j        ],
       [0.        +0.j        , 0.70710678-0.70710678j]])},
 'K4': {'angle (frac)': (0, 1, 0.0, 0.0, 0.0),
        'angle (π)': 0.0,
        'axis': (1, 0, 0),
        'global_phase': (1+0j),
        'matrix': array([[1.+0.j, 0.+0.j],
       [0.+0.j, 1.+0.j]])},
 'coefficients': {'t_x': 2